In [1]:
import torch
from detr.data import CocoDataset, get_val_transforms

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
IMAGE_SIZE = 608

val_dataset = CocoDataset(
    dataset_root = "/media/bryan/ssd01/fiftyone/coco-2017",
    split = "validation",
    transform = get_val_transforms(resize_size=IMAGE_SIZE)
)

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [4]:
data = val_dataset[10]